**Import**

In [8]:

from tinyec.ec import SubGroup, Curve
from util import generate_prime_number, add_points, mul_point, modinv, mod_exp
import secrets
import random


In [9]:


# ============================================================
# 🧩 1️⃣ HÀM HỖ TRỢ
# ============================================================



def legendre_symbol(a, p):
    """Ký hiệu Legendre: 1 nếu a là bình phương mod p, -1 nếu không"""
    ls = pow(a, (p - 1) // 2, p)
    return -1 if ls == p - 1 else ls

def mod_sqrt(a, p):
    """Tìm căn bậc hai mod p (khi p ≡ 3 mod 4)"""
    if p % 4 == 3:
        return pow(a, (p + 1) // 4, p)
    else:
        return None  # chỉ xử lý p mod 4 == 3 cho nhanh

# ============================================================
# 🧠 2️⃣ TÌM ĐƯỜNG CONG HỢP LỆ VỚI p LỚN
# ============================================================

def find_valid_curve(bits=64):
    """Sinh p lớn, a,b ngẫu nhiên, tìm G hợp lệ"""
    while True:
        # Sinh số nguyên tố p 64 bit, p ≡ 3 (mod 4) để tính sqrt nhanh
        p = generate_prime_number(64)

        a = random.randrange(1, p)
        b = random.randrange(1, p)

        # Tìm điểm G hợp lệ (chọn x ngẫu nhiên)
        for _ in range(2000):
            x = random.randrange(2, p)
            rhs = (x**3 + a*x + b) % p
            if legendre_symbol(rhs, p) == 1:  # tồn tại căn bậc hai
                y = mod_sqrt(rhs, p)
                if y is not None:
                    return p, a, b, (x, y)


# ============================================================
# 🔐 4️⃣ SINH ĐƯỜNG CONG VÀ KHÓA
# ============================================================

p, a, b, G = find_valid_curve(64)
print(f"✅ p = {p}")
print(f"✅ Đường cong: y² = x³ + {a}x + {b} (mod {p})")
print(f"✅ G = {G}")

d = random.randint(2, p-2)      # Khóa bí mật
Q = mul_point(d, G, a, p)       # Khóa công khai
print(f"\n🔹 Khóa bí mật d = {d}")
print(f"🔹 Khóa công khai Q = {Q}")

# ============================================================
# ✍️ 5️⃣ KÝ
# ============================================================

message = 68
h_val = (message + 5) % p
k = 7
R = mul_point(k, G, a, p)
r = R[0] % p
k_inv = mod_exp(k, p-2,p)
s = (k_inv * (h_val + d * r)) % p

print(f"\n✍️ Ký:")
print(f"  h(x) = {h_val}")
print(f"  k = {k}")
print(f"  R = {R}")
print(f"  (r, s) = ({r}, {s})")

# ============================================================
# 🔍 6️⃣ VERIFY
# ============================================================

w = mod_exp(s,p-2, p)
u1 = (h_val * w) % p
u2 = (r * w) % p
P = add_points(mul_point(u1, G, a, p), mul_point(u2, Q, a, p), a, p)

print(f"\n🔍 Verify:")
print(f"  u1 = {u1}, u2 = {u2}")
print(f"  P = {P}")

if P != ("O", "O") and (P[0] % p) == r:
    print("✅ Chữ ký hợp lệ (Valid Signature)")
else:
    print("❌ Chữ ký không hợp lệ (Invalid Signature)")


✅ p = 13285020848308471219
✅ Đường cong: y² = x³ + 12042803703396870632x + 13005512341055249220 (mod 13285020848308471219)
✅ G = (5716160276262140976, 1293214199951857155)


KeyboardInterrupt: 